In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))




In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [2]:
# Set dataset path
dataset_path = "/kaggle/input/chest-xray-pneumonia/chest_xray"

# Image parameters
IMG_SIZE = 150
BATCH_SIZE = 32

# Data augmentation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load training data
train_generator = datagen.flow_from_directory(
    dataset_path + "/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training"
)

# Load validation data
val_generator = datagen.flow_from_directory(
    dataset_path + "/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation"
)


Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.


In [5]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (Normal/Pneumonia)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss="binary_crossentropy",
              metrics=["accuracy"])


In [6]:
# Train for only 5 epochs
model.fit(train_generator, validation_data=val_generator, epochs=5)

# Save the model
model.save("pneumonia_model.h5")


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


131/131 ━━━━━━━━━━━━━━━━━━━━ 126s 920ms/step - accuracy: 0.7894 - loss: 0.4686 - val_accuracy: 0.9281 - val_loss: 0.1874
Epoch 2/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 112s 837ms/step - accuracy: 0.9479 - loss: 0.1366 - val_accuracy: 0.9578 - val_loss: 0.1317
Epoch 3/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 113s 839ms/step - accuracy: 0.9567 - loss: 0.1297 - val_accuracy: 0.9569 - val_loss: 0.1209
Epoch 4/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 111s 832ms/step - accuracy: 0.9640 - loss: 0.0919 - val_accuracy: 0.9549 - val_loss: 0.1189
Epoch 5/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 118s 885ms/step - accuracy: 0.9717 - loss: 0.0738 - val_accuracy: 0.9444 - val_loss: 0.1677


In [7]:
# Load test data
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    dataset_path + "/test",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

# Evaluate model performance
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")


Found 624 images belonging to 2 classes.
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 478ms/step - accuracy: 0.7915 - loss: 0.7351
Test Accuracy: 0.7901
